In [1]:
import pandas as pd
import pickle
import numpy as np
from collections import Counter
from gensim import corpora
import re

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [133]:
# to communicate with google spreadsheet...
import gspread
from gspread_dataframe import get_as_dataframe
from gspread_dataframe import set_with_dataframe
from google.oauth2 import service_account # based on google-auth library
import sddk

s = sddk.cloudSession("sciencedata.dk")
# establish connection with gogglesheets...
file_data = s.read_file("https://sciencedata.dk/files/ServiceAccountsKey.json", "dict") # or load it from a local storage: json.load(open("../../ServiceAccountsKey.json", "r"))
credentials = service_account.Credentials.from_service_account_info(file_data)
gc = gspread.Client(auth=credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']))
mops_data = gc.open_by_url("https://docs.google.com/spreadsheets/d/1VbCIAJssHKV9hlRTwzVFfm40CGnHesq53KXjv2qy4OM/edit?usp=sharing")

endpoint variable has been configured to: https://sciencedata.dk/files/


In [3]:
jstor_df = pd.read_feather("../data/large_files/jstor_df_v1.feather")

In [4]:
trigramCount_cleaned_dict = pickle.load(open("../data/large_files/trigramCount_cleaned_dict.pickle", "rb"))

In [134]:
to_replace = {
    "christian" : "Christian",
    "Biblical" : "biblical",
    "Church" : "church",
    "Faith" : "faith",
    "Apostle" : "apostle",
    "american" : "American",
    "jewish" : "Jewish",
    "old Testament" : "Old Testament",
    "Paul St" : "Paul saint",
    "Lord" : "lord"
}
to_replace_keys = list(to_replace.keys())

In [136]:
key = "Church"
re.sub(key, to_replace[key], "Church Jesus")

'church Jesus'

In [ ]:
def clean_trigram_string(trigram_string):
    for key in to_replace_keys:
        if key in trigram_string:
            trigram_string = re.sub(key, to_replace[key], trigram_string)
    return trigram_string

In [ ]:
for id, data in trigramCount_cleaned_dict.items():
    data.items()


# Preprocesing for embeddings etc.


In [2]:
unigrams_merged_cleaned = pickle.load(open("../data/large_files/unigrams_merged_cleaned.pickle", "rb"))

In [3]:
#
types_N = len(unigrams_merged_cleaned)
types_N

1349906

In [4]:
tokens_N = sum([tup[1] for tup in unigrams_merged_cleaned.items()])
tokens_N

71637349

In [5]:
threshold = 50
unigrams_merged_thresh = dict([tup for tup in unigrams_merged_cleaned.items() if tup[1] >= threshold])
types_N_thres = len(unigrams_merged_thresh)
types_N_thres

45412

In [6]:
tokens_N_thres = sum([tup[1] for tup in unigrams_merged_thresh.items()])
tokens_N_thres

67711862

In [7]:
print(np.round(types_N_thres / types_N * 100, 2))
print(np.round(tokens_N_thres / tokens_N * 100, 2))

3.36
94.52


In [8]:
unigrams_sorted_tups = sorted(list(unigrams_merged_thresh.items()), key = lambda tup: tup[1], reverse=True)
unigrams_sorted_tups[:10]

[('have', 1104349),
 ('God', 497732),
 ('do', 464566),
 ('Paul', 407736),
 ('church', 378646),
 ('see', 373526),
 ('Christian', 319494),
 ('other', 294611),
 ('new', 286333),
 ('Jesus', 281420)]

In [9]:
vocabulary = [tup[0] for tup in unigrams_sorted_tups]

In [10]:
dictionary = corpora.Dictionary([vocabulary])

In [11]:
dictionary.token2id["Christ"]

3026

In [16]:
dictionary[3026]

'Christ'

In [17]:
len(dictionary)

45412

In [18]:
pickle.dump(dictionary, open("../data/dictionary_main.pickle", "wb"))

In [137]:
dictionary.doc2bow(["Jesus", "Christ", "nonsenseword"])

[(3026, 1), (7872, 1)]

In [138]:
def clean_trigram_string(trigram_string):
    for key in to_replace_keys:
        if key in trigram_string:
            trigram_string = re.sub(key, to_replace[key], trigram_string)
    return trigram_string

In [148]:
%%time
trigramCount_bows = {}
for id, data in list(trigramCount_cleaned_dict.items()):
    data = dict([(clean_trigram_string(trigram), count) for trigram, count in data.items() if len(trigram.split()) > 1])
    data_bows = []
    for trigram, count in data.items():
        bow = dictionary.doc2bow(trigram.split())
        if len(bow) > 1:
            data_bows.extend([bow] * count)
    trigramCount_bows[id] = data_bows

CPU times: user 4min 48s, sys: 7min 45s, total: 12min 33s
Wall time: 45min 38s


In [150]:
pickle.dump(trigramCount_bows, open("../data/large_files/trigramCount_bows.pickle", "wb"))